# Preparación de los Datos



In [64]:
import pandas as pd
from pathlib import Path

# Librerías de scikit learn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
# Librerías para crear gráficos
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [123]:

with open("/Users/juanantoniohidalgoperez/Downloads/Material complementario-4/column_names.txt", "r") as file:
    nombres_columnas = file.read().splitlines()

datos = pd.read_csv("/Users/juanantoniohidalgoperez/Downloads/Material complementario-4/gene_expression.csv", delimiter=";", dtype=str)

datos.columns = nombres_columnas


columnas_expresion_genica = datos.columns[0:]  
for columna in columnas_expresion_genica:
    datos[columna] = pd.to_numeric(datos[columna].str.replace('.', ''), errors='coerce')


archivo2 = pd.read_csv("/Users/juanantoniohidalgoperez/Downloads/Material complementario-4/classes.csv", delimiter=";", dtype=str)


resultado = pd.concat([archivo2, datos], axis=1)

resultado.to_csv("resultado_fusion.csv", index=False)


resultado_final_final = pd.read_csv("resultado_fusion.csv")
resultado_final_final.head(10)



,Sample,Classes,DUOXA1,INTU,UMOD,STXBP4,PARN,TSC22D4,TTLL5,ZNF880,...,NPTN,BCAR3,PVALB,PTPN11,C19orf33,GASK1A,TMEM114,UCN,RAB25,TTC31
0,sample_0,CHC,1.040564e+11,9.816923e+11,1.069198e+11,7.619750e+11,8.349710e+11,1.029648e+11,9.360250e+11,9.148949e+11,...,1.004394e+11,1.085426e+11,9.678533e+11,1.174790e+11,3.236583e+11,4.828642e+11,7.183635e+11,5.030870e+10,0.0,1.108191e+11
1,sample_1,CGC,1.160917e+11,9.701637e+11,1.171037e+11,8.883569e+11,7.627724e+11,8.245710e+11,5.228084e+11,1.065897e+11,...,4.525954e+11,1.045707e+11,8.876802e+11,0.000000e+00,7.965978e+11,5.897010e+11,8.324109e+10,7.965978e+11,0.0,1.019530e+11
2,sample_2,CHC,1.192217e+11,1.030129e+11,9.662613e+11,9.003394e+11,6.772823e+11,7.984322e+11,5.218564e+11,1.010282e+11,...,1.267356e+11,1.135403e+11,8.916787e+11,0.000000e+00,1.931418e+11,7.624240e+11,1.074984e+11,6.469165e+11,0.0,1.009819e+11
3,sample_3,CHC,1.081117e+11,1.027564e+11,9.582229e+11,1.037347e+11,7.510693e+11,9.654743e+11,7.597956e+11,9.184303e+11,...,2.612801e+11,1.097804e+10,8.692664e+11,0.000000e+00,6.254220e+11,8.100505e+10,1.004159e+10,6.306973e+11,0.0,1.054990e+11
4,sample_4,CFB,1.142512e+11,1.022029e+11,1.070002e+11,8.862876e+11,8.672574e+11,8.369619e+11,7.499088e+11,9.803579e+11,...,2.002018e+11,1.080346e+11,9.082162e+11,5.154097e+11,4.718789e+11,7.951040e+11,9.663948e+11,8.570360e+11,0.0,1.049425e+11
5,sample_5,CHC,1.091280e+11,1.103952e+11,1.137621e+11,9.243157e+11,8.792026e+11,1.187387e+11,8.478867e+11,9.485839e+10,...,7.035437e+11,1.099956e+11,9.055242e+11,9.196833e+11,6.131423e+11,8.080700e+11,1.005026e+11,6.570924e+11,0.0,1.036070e+11
6,sample_6,AGH,1.164686e+11,1.076322e+11,1.103536e+11,9.687759e+11,8.931568e+11,9.359045e+11,8.437769e+11,9.624989e+11,...,2.304861e+11,1.099262e+11,8.899454e+11,1.481041e+11,4.602237e+11,7.293315e+11,1.061521e+11,6.110113e+10,0.0,9.932206e+11
7,sample_7,CHC,1.096611e+11,1.109995e+11,9.108621e+11,9.621543e+11,7.439739e+11,9.739215e+11,5.036147e+10,1.047429e+11,...,3.559296e+11,9.725138e+11,9.482854e+11,6.353365e+11,7.428201e+11,8.170611e+11,9.975174e+11,5.963724e+11,0.0,1.058470e+11
8,sample_8,CFB,1.089298e+11,9.465795e+11,1.082045e+11,1.021625e+11,7.430243e+11,7.652221e+11,7.001566e+10,9.730042e+10,...,3.244278e+11,1.153031e+11,9.016900e+10,1.204141e+11,4.793475e+11,9.712211e+11,1.076531e+11,6.910673e+11,0.0,1.037909e+11
9,sample_9,CHC,1.103010e+11,1.042331e+11,9.728570e+11,1.020296e+11,8.980799e+10,9.284176e+10,8.130926e+11,9.622795e+11,...,3.192147e+11,1.104714e+11,9.089170e+11,5.144000e+11,4.850309e+11,6.913189e+11,1.002872e+11,6.028138e+11,0.0,1.079606e+10


In [103]:
resultado_final_final.shape

(801, 502)

In [104]:
total_nan = resultado_final_final.isna().sum().sum()

print(f"Número total de valores nulos: {total_nan}")

Número total de valores nulos: 500


In [110]:
data_clean = resultado_final_final.dropna(axis=1, how="any")
data_clean.shape

(801, 2)

In [108]:
resultado_final_final.dtypes

Sample      object
Classes     object
DUOXA1     float64
INTU       float64
UMOD       float64
            ...   
GASK1A     float64
TMEM114    float64
UCN        float64
RAB25      float64
TTC31      float64
Length: 502, dtype: object

In [131]:
data_clean = resultado_final_final.drop(["Sample", "Classes"], axis=1)

imputador_knn = KNNImputer(n_neighbors=5)  # Puedes ajustar el número de vecinos (n_neighbors)

# Realizar la imputación
datos_imputados = pd.DataFrame(imputador_knn.fit_transform(data_clean), columns=data_clean.columns)

# Guardar los datos imputados en un nuevo archivo CSV
datos_imputados.to_csv("datos_imputados.csv", index=False)



datos_imputados.head(10)


,DUOXA1,INTU,UMOD,STXBP4,PARN,TSC22D4,TTLL5,ZNF880,SPATA1,PLIN3,...,NPTN,BCAR3,PVALB,PTPN11,C19orf33,GASK1A,TMEM114,UCN,RAB25,TTC31
0,1.040564e+11,9.816923e+11,1.069198e+11,7.619750e+11,8.349710e+11,1.029648e+11,9.360250e+11,9.148949e+11,7.210837e+11,0.000000e+00,...,1.004394e+11,1.085426e+11,9.678533e+11,1.174790e+11,3.236583e+11,4.828642e+11,7.183635e+11,5.030870e+10,0.0,1.108191e+11
1,1.160917e+11,9.701637e+11,1.171037e+11,8.883569e+11,7.627724e+11,8.245710e+11,5.228084e+11,1.065897e+11,7.830509e+11,0.000000e+00,...,4.525954e+11,1.045707e+11,8.876802e+11,0.000000e+00,7.965978e+11,5.897010e+11,8.324109e+10,7.965978e+11,0.0,1.019530e+11
2,1.192217e+11,1.030129e+11,9.662613e+11,9.003394e+11,6.772823e+11,7.984322e+11,5.218564e+11,1.010282e+11,8.319085e+11,0.000000e+00,...,1.267356e+11,1.135403e+11,8.916787e+11,0.000000e+00,1.931418e+11,7.624240e+11,1.074984e+11,6.469165e+11,0.0,1.009819e+11
3,1.081117e+11,1.027564e+11,9.582229e+11,1.037347e+11,7.510693e+11,9.654743e+11,7.597956e+11,9.184303e+11,7.279471e+11,1.275841e+11,...,2.612801e+11,1.097804e+10,8.692664e+11,0.000000e+00,6.254220e+11,8.100505e+10,1.004159e+10,6.306973e+11,0.0,1.054990e+11
4,1.142512e+11,1.022029e+11,1.070002e+11,8.862876e+11,8.672574e+11,8.369619e+11,7.499088e+11,9.803579e+11,7.169725e+11,0.000000e+00,...,2.002018e+11,1.080346e+11,9.082162e+11,5.154097e+11,4.718789e+11,7.951040e+11,9.663948e+11,8.570360e+11,0.0,1.049425e+11
5,1.091280e+11,1.103952e+11,1.137621e+11,9.243157e+11,8.792026e+11,1.187387e+11,8.478867e+11,9.485839e+10,6.149656e+11,2.042924e+11,...,7.035437e+11,1.099956e+11,9.055242e+11,9.196833e+11,6.131423e+11,8.080700e+11,1.005026e+11,6.570924e+11,0.0,1.036070e+11
6,1.164686e+11,1.076322e+11,1.103536e+11,9.687759e+11,8.931568e+11,9.359045e+11,8.437769e+11,9.624989e+11,6.812883e+11,4.418022e+11,...,2.304861e+11,1.099262e+11,8.899454e+11,1.481041e+11,4.602237e+11,7.293315e+11,1.061521e+11,6.110113e+10,0.0,9.932206e+11
7,1.096611e+11,1.109995e+11,9.108621e+11,9.621543e+11,7.439739e+11,9.739215e+11,5.036147e+10,1.047429e+11,1.062214e+11,0.000000e+00,...,3.559296e+11,9.725138e+11,9.482854e+11,6.353365e+11,7.428201e+11,8.170611e+11,9.975174e+11,5.963724e+11,0.0,1.058470e+11
8,1.089298e+11,9.465795e+11,1.082045e+11,1.021625e+11,7.430243e+11,7.652221e+11,7.001566e+10,9.730042e+10,8.662626e+11,0.000000e+00,...,3.244278e+11,1.153031e+11,9.016900e+10,1.204141e+11,4.793475e+11,9.712211e+11,1.076531e+11,6.910673e+11,0.0,1.037909e+11
9,1.103010e+11,1.042331e+11,9.728570e+11,1.020296e+11,8.980799e+10,9.284176e+10,8.130926e+11,9.622795e+11,7.208888e+11,0.000000e+00,...,3.192147e+11,1.104714e+11,9.089170e+11,5.144000e+11,4.850309e+11,6.913189e+11,1.002872e+11,6.028138e+11,0.0,1.079606e+10


In [132]:
data_clean.shape
total_nan = datos_imputados.isna().sum().sum()

print(f"Número total de valores nulos: {total_nan}")

Número total de valores nulos: 0


In [133]:
# Estandarización de los datos
scaler = StandardScaler() # Crea una instancia de StandardScaler

# Estandariza los datos utilizando la instancia de StandarScaler
data_standardized = scaler.fit_transform(datos_imputados)

# Muestra los datos estandarizados
data_standardized[10]

array([ 2.32138476e-01, -5.10178549e-01, -9.48100653e-01,  6.61988690e-01,
        3.82276932e-01, -9.66099654e-01, -5.23489645e-01, -9.61785754e-01,
        6.29134773e-02, -2.33070069e-01,  1.74206121e-01, -6.94178229e-01,
        1.37131126e-01,  2.00352638e-01,  6.84119818e-01, -5.55813019e-01,
       -1.06994447e-01, -8.95827235e-01,  6.64557680e-01,  5.20905452e-01,
        6.40062831e-01,  5.13864056e-01, -4.47184150e-01,  2.87613184e-01,
        4.88802652e-01,  3.72119974e-01,  3.72854791e-01,  2.95552966e-02,
        5.58635835e-01,  9.49484779e-01, -3.16811848e-01,  1.68246381e-01,
        2.79536090e-01, -1.91575098e-01, -1.38434515e+00,  1.40142339e-01,
        7.34801395e-01, -5.19858704e-01,  8.15567379e-02, -1.44685474e-01,
        2.27297703e-01,  3.02288638e-01,  1.61479081e-01,  6.83659320e-01,
        3.74345183e-01, -1.45068908e-01,  2.50930630e-01, -4.78599681e-01,
       -4.95806497e-01,  5.17355275e-01, -1.50422755e+00,  1.12233146e+00,
       -1.54789891e+00, -

# Uso de PCA

In [134]:
# PCA para 2 componentes
pca_2d = PCA(n_components=2)
components_2d = pca_2d.fit_transform(data_standardized)

# PCA para 3 componentes
pca_3d = PCA(n_components=3)
components_3d = pca_3d.fit_transform(data_standardized)

In [135]:
# Varianza explicada para 2 y 3 componentes
explained_variance_2d = pca_2d.explained_variance_ratio_
explained_variance_3d = pca_3d.explained_variance_ratio_

# Cálculo de la varianza explicada acumulada
cumulative_variance_2d = explained_variance_2d.sum()
cumulative_variance_3d = explained_variance_3d.sum()

# Mostrar resultados
print(f"Varianza explicada PCA2: {explained_variance_2d}")
print(f"Varianza acumulada PCA2: {cumulative_variance_2d}")
print(f"Varianza explicada PCA3: {explained_variance_3d}")
print(f"Varianza acumulada PCA3: {cumulative_variance_3d}")

Varianza explicada PCA2: [0.03420861 0.02925245]
Varianza acumulada PCA2: 0.06346106361489558
Varianza explicada PCA3: [0.03420861 0.02925245 0.02442462]
Varianza acumulada PCA3: 0.08788568008864374
